In [1]:
import jax
from jax import vmap
import jax.numpy as jnp

In [2]:
def f(x, y):
    return jnp.array([[x[0], x[1]], [y[0], y[1]]])

def f2(x, y):
    return jnp.array([
        [1000*x[0] + 100*x[1] + 10*x[0] + x[1], 
         1000*x[0] + 100*x[1] + 10*y[0] + y[1]],
        [1000*y[0] + 100*y[1] + 10*x[0] + x[1], 
         1000*y[0] + 100*y[1] + 10*y[0] + y[1]],
    ])

def f3(x, y):
    return jnp.array([
        [1000 + 100*x + 10 + y, 
         1000 + 100*x + 20 + y],
        [2000 + 100*x + 10 + y, 
         2000 + 100*x + 20 + y],
    ])
def gxf(x, y):
    return jnp.array([
        [100 + 10*x + y],  
        [200 + 10*x + y]
    ])
def gfx(x, y):
    return jnp.array([
        100 * x + 10 + y, 
        100 * x + 20 + y
    ])

In [3]:
x = jnp.array([1., 1.])
y = jnp.array([2., 1.])

ERROR! Session/line number was not unique in database. History logging moved to new session 2


In [4]:
f3(2,1)

Array([[1211, 1221],
       [2211, 2221]], dtype=int32)

# Testing `vmap`

In [5]:
#xx = jnp.arange(1., 11.).reshape(5,2)
#yy = jnp.arange(3., 13.).reshape(5,2)

# xx = jnp.array([
#     [1, 1],
#     [2, 1],
#     [1, 2],
#     [2, 2],
#     [1, 3],
#     [2, 3],
#     [1, 4],
#     [2, 4],
# ])
# yy = xx.copy()

xx = jnp.arange(1, 1 + 5)
yy = jnp.arange(1, 1 + 5)

## `f3` version: the Gram matrix

In [6]:
#fv3 = vmap(f3, in_axes=(0, None))
fv3 = vmap(f3, in_axes=(0, None), out_axes=(0,2))

In [7]:
fv3 = vmap(f3, in_axes=(0, None), out_axes=0)
fvv3 = vmap(fv3, in_axes=(None, 0), out_axes=0)
#fvv3 = vmap(f3, in_axes=(0,0), out_axes=(0,0)) # This seems to work!
#print("fv:\n", fv3(xx, 1))
#print("fvv:\n", fvv3(xx, yy))
print("fv concatenated:\n", jnp.concatenate(fv3(xx[:2], 1), 0))
print("fvv concatenated:\n", jnp.concatenate(jnp.concatenate(fv3(xx[:2], yy[:2]), 2), 0))

fv concatenated:
 [[1111 1121]
 [2111 2121]
 [1211 1221]
 [2211 2221]]
fvv concatenated:
 [[1111 1112 1211 1212]
 [1121 1122 1221 1222]
 [2111 2112 2211 2212]
 [2121 2122 2221 2222]]


## `gxf`/ `gfx` version: cross-covariances between function value predictions and derivative observations

In [69]:
gxfv = vmap(gxf, (0, None), 0)
gfxv = vmap(gfx, (0, None), 1)
print("gxfv:\n", gxfv(xx[:2], 1))
print("gfxv:\n", gfxv(xx[:2], 1))
print("gxfv concatenated:\n", jnp.concatenate(gxfv(xx[:2], 1), 0))
print("gfxv concatenated:\n", jnp.concatenate(gfxv(xx[:2], 1), 0))

gxfv:
 [[[[111. 121.]]

  [[211. 221.]]]


 [[[131. 141.]]

  [[231. 241.]]]]
gfxv:
 [[[111. 211.]
  [311. 411.]]

 [[121. 221.]
  [321. 421.]]]
gxfv concatenated:
 [[[111. 121.]]

 [[211. 221.]]

 [[131. 141.]]

 [[231. 241.]]]
gfxv concatenated:
 [[111. 211.]
 [311. 411.]
 [121. 221.]
 [321. 421.]]


In [70]:
gxfvv = vmap(gxfv, (None, 0), 1)
gfxvv = vmap(gfxv, (None, 0), 1)
print("gxfvv:\n", gxfvv(xx[:2], yy[:2]))
print("gfxvv:\n", gfxvv(xx[:2], yy[:2]))
print("gxfvv concatenated:\n", jnp.concatenate(gxfvv(xx[:2], yy[:2]), 0))
print("gfxvv concatenated:\n", jnp.concatenate(gfxvv(xx[:2], yy[:2]), 0))

gxfvv:
 [[[[[111. 122.]]

   [[211. 222.]]]


  [[[113. 124.]]

   [[213. 224.]]]]



 [[[[131. 142.]]

   [[231. 242.]]]


  [[[133. 144.]]

   [[233. 244.]]]]]
gfxvv:
 [[[[111. 212.]
   [311. 412.]]

  [[113. 214.]
   [313. 414.]]]


 [[[121. 222.]
   [321. 422.]]

  [[123. 224.]
   [323. 424.]]]]
gxfvv concatenated:
 [[[[111. 122.]]

  [[211. 222.]]]


 [[[113. 124.]]

  [[213. 224.]]]


 [[[131. 142.]]

  [[231. 242.]]]


 [[[133. 144.]]

  [[233. 244.]]]]
gfxvv concatenated:
 [[[111. 212.]
  [311. 412.]]

 [[113. 214.]
  [313. 414.]]

 [[121. 222.]
  [321. 422.]]

 [[123. 224.]
  [323. 424.]]]


## Older `gxf` test

In [71]:
gxfv = vmap(gxf, (0, None), 0)
gxfvv = vmap(gxfv, (None, 0), 1)
print("gxfv:\n", gxfv(xx[:2], 1))

gxfv:
 [[[[111. 121.]]

  [[211. 221.]]]


 [[[131. 141.]]

  [[231. 241.]]]]


In [72]:
jnp.concatenate(fv3(xx[:2], 1), 0)

Array([[[1111., 1211.],
        [1121., 1221.]],

       [[2111., 2211.],
        [2121., 2221.]],

       [[1311., 1411.],
        [1321., 1421.]],

       [[2311., 2411.],
        [2321., 2421.]]], dtype=float32)

In [73]:
fvv3 = vmap(fv3, in_axes=(None, 0))

In [74]:
t = fvv3(xx,yy)

In [75]:
jnp.concatenate(jnp.concatenate(t, axis=2), axis=0)

Array([[[1111., 1212.],
        [1121., 1222.],
        [1113., 1214.],
        [1123., 1224.],
        [1115., 1216.],
        [1125., 1226.],
        [1117., 1218.],
        [1127., 1228.],
        [1119., 1220.],
        [1129., 1230.]],

       [[2111., 2212.],
        [2121., 2222.],
        [2113., 2214.],
        [2123., 2224.],
        [2115., 2216.],
        [2125., 2226.],
        [2117., 2218.],
        [2127., 2228.],
        [2119., 2220.],
        [2129., 2230.]],

       [[1311., 1412.],
        [1321., 1422.],
        [1313., 1414.],
        [1323., 1424.],
        [1315., 1416.],
        [1325., 1426.],
        [1317., 1418.],
        [1327., 1428.],
        [1319., 1420.],
        [1329., 1430.]],

       [[2311., 2412.],
        [2321., 2422.],
        [2313., 2414.],
        [2323., 2424.],
        [2315., 2416.],
        [2325., 2426.],
        [2317., 2418.],
        [2327., 2428.],
        [2319., 2420.],
        [2329., 2430.]],

       [[1511., 1612.],
        

## Older tests with `reshape`

In [76]:
fv = vmap(f, in_axes=(0, None))

In [77]:
fv(xx, y)

Array([[[ 1.,  2.],
        [ 2.,  1.]],

       [[ 3.,  4.],
        [ 2.,  1.]],

       [[ 5.,  6.],
        [ 2.,  1.]],

       [[ 7.,  8.],
        [ 2.,  1.]],

       [[ 9., 10.],
        [ 2.,  1.]]], dtype=float32)

In [78]:
fvv = vmap(fv, in_axes=(None, 0))

In [79]:
fvv(xx, yy).shape

(5, 5, 2, 2)

In [80]:
fvv(xx, yy)

Array([[[[ 1.,  2.],
         [ 1.,  2.]],

        [[ 3.,  4.],
         [ 1.,  2.]],

        [[ 5.,  6.],
         [ 1.,  2.]],

        [[ 7.,  8.],
         [ 1.,  2.]],

        [[ 9., 10.],
         [ 1.,  2.]]],


       [[[ 1.,  2.],
         [ 3.,  4.]],

        [[ 3.,  4.],
         [ 3.,  4.]],

        [[ 5.,  6.],
         [ 3.,  4.]],

        [[ 7.,  8.],
         [ 3.,  4.]],

        [[ 9., 10.],
         [ 3.,  4.]]],


       [[[ 1.,  2.],
         [ 5.,  6.]],

        [[ 3.,  4.],
         [ 5.,  6.]],

        [[ 5.,  6.],
         [ 5.,  6.]],

        [[ 7.,  8.],
         [ 5.,  6.]],

        [[ 9., 10.],
         [ 5.,  6.]]],


       [[[ 1.,  2.],
         [ 7.,  8.]],

        [[ 3.,  4.],
         [ 7.,  8.]],

        [[ 5.,  6.],
         [ 7.,  8.]],

        [[ 7.,  8.],
         [ 7.,  8.]],

        [[ 9., 10.],
         [ 7.,  8.]]],


       [[[ 1.,  2.],
         [ 9., 10.]],

        [[ 3.,  4.],
         [ 9., 10.]],

        [[ 5.,  6.],
   

In [81]:
fvv(xx, yy).reshape(10, 10)

Array([[ 1.,  2.,  1.,  2.,  3.,  4.,  1.,  2.,  5.,  6.],
       [ 1.,  2.,  7.,  8.,  1.,  2.,  9., 10.,  1.,  2.],
       [ 1.,  2.,  3.,  4.,  3.,  4.,  3.,  4.,  5.,  6.],
       [ 3.,  4.,  7.,  8.,  3.,  4.,  9., 10.,  3.,  4.],
       [ 1.,  2.,  5.,  6.,  3.,  4.,  5.,  6.,  5.,  6.],
       [ 5.,  6.,  7.,  8.,  5.,  6.,  9., 10.,  5.,  6.],
       [ 1.,  2.,  7.,  8.,  3.,  4.,  7.,  8.,  5.,  6.],
       [ 7.,  8.,  7.,  8.,  7.,  8.,  9., 10.,  7.,  8.],
       [ 1.,  2.,  9., 10.,  3.,  4.,  9., 10.,  5.,  6.],
       [ 9., 10.,  7.,  8.,  9., 10.,  9., 10.,  9., 10.]], dtype=float32)

# Understanding dimensions in `vmap` and `concatenate`

## `vmap`

In [82]:
# Test data

xx = jnp.arange(1, 1 + 5)
yy = jnp.arange(1, 1 + 5)

`out_axes` is where the looped-over axies will be put:

In [83]:
for i in range(3):
    print(f"out_axes={i}:", vmap(f3, in_axes=(0, None), out_axes=i)(xx[:3], 1).shape)

out_axes=0: (3, 2, 2)
out_axes=1: (2, 3, 2)
out_axes=2: (2, 2, 3)


And with the actual output:

In [84]:
for i in range(3):
    print(f"out_axes={i}:\n", vmap(f3, in_axes=(0, None), out_axes=i)(xx[:3], 1))

out_axes=0:
 [[[1111 1121]
  [2111 2121]]

 [[1211 1221]
  [2211 2221]]

 [[1311 1321]
  [2311 2321]]]
out_axes=1:
 [[[1111 1121]
  [1211 1221]
  [1311 1321]]

 [[2111 2121]
  [2211 2221]
  [2311 2321]]]
out_axes=2:
 [[[1111 1211 1311]
  [1121 1221 1321]]

 [[2111 2211 2311]
  [2121 2221 2321]]]


**Take-away:** For $k(x_i, x_j)$, we want $x_i$ to be mapped to the rows of the resulting matrix, and $x_j$ to the columns. The innermost `vmap` works on $x_i$, so we want this output to be put at axis 0 of the final matrix. The outermost `vmap` works on $x_j$, which should be put at axis 1 of the final matrix. Thus, this seems to be what we want:

In [93]:
fv3 = vmap(f3, in_axes=(0, None), out_axes=0)
fvv3 = vmap(fv3, in_axes=(None, 0), out_axes=1)

fv_mat = fv3(xx[:3], 1)
fvv_mat = fvv3(xx[:3], yy[:4])
fvv_mat_a = vmap(lambda x: vmap(lambda y: f3(x,y))(yy[:4]))(xx[:3])

print("fv shape:", fv_mat.shape)
print("fvv shape:", fvv_mat.shape)
print("fvv_a shape:", fvv_mat_a.shape)

fv shape: (3, 2, 2)
fvv shape: (3, 4, 2, 2)
fvv_a shape: (3, 4, 2, 2)


In [92]:
fvv_mat == fvv_mat_a

Array([[[[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]]],


       [[[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]]],


       [[[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]]]], dtype=bool)

### For constructing the cross-covariances

#### Constructing $\mathrm{cov}(y^m, \omega^{i, n})$

Looping over the `out_axes` for the block below the Gram matrix block (i.e., $\mathrm{cov}(y^m, \omega^{i, n}) = [\mathrm{cov}(y^m, \omega^{1, n})\quad \mathrm{cov}(y^m, \omega^{2, n})]$):

In [ ]:
for i in range(2):
    gfx_vec = vmap(gfx, in_axes=(None, 0), out_axes=i)(1, xx[:3])
    print(f"out_axes={i}: {gfx_vec.shape}\n", gfx_vec)

As before, `out_axes=0` is what we want. 

Using the intuition from constructing the Gram matrix, we want the first input to the covariance function to be put at axis 0 and the second input to be put at axis 1. 

But this is incorrect! It has to be axis 0 both times, then a concatenation across axis 0 gives the right matrix.

In [ ]:
gfx_v = vmap(gfx, in_axes=(0, None), out_axes=0)
gfx_vv = vmap(gfx_v, in_axes=(None, 0), out_axes=0)

gfx_vec = gfx_vv(xx[:1], yy[:4])
gfx_mat = gfx_vv(xx[:3], yy[:4])

print("Vector shape:", gfx_vec.shape)
print(gfx_vec)
print()
print("Matrix shape:", gfx_mat.shape)
print(gfx_mat)
print()

#### Constructing $\mathrm{cov}(\omega^{j, m}, y^m)$

Looping over the `out_axes` for the block to the right of the Gram matrix block (i.e., $\mathrm{cov}(\omega^{j, m}, y^n) = [\mathrm{cov}(\omega^{1, m}, y^n)\quad \mathrm{cov}(\omega^{2, m}, y^n)]^\top$):

In [9]:
for i in range(2):
    gxf_vec = vmap(gxf, in_axes=(0, None), out_axes=i)(xx[:3], 1)
    print(f"out_axes={i}: {gxf_vec.shape}\n", gxf_vec)

out_axes=0: (3, 2, 1)
 [[[111]
  [211]]

 [[121]
  [221]]

 [[131]
  [231]]]
out_axes=1: (2, 3, 1)
 [[[111]
  [121]
  [131]]

 [[211]
  [221]
  [231]]]


In [92]:
gxf_v = vmap(gxf, in_axes=(0, None), out_axes=0)
gxf_vv = vmap(gxf_v, in_axes=(None, 0), out_axes=0)

gxf_vec = gxf_vv(xx[:1], yy[:4])
gxf_mat = gxf_vv(xx[:3], yy[:4])

print("Vector shape:", gxf_vec.shape)
print(gxf_vec)
print()
print("Matrix shape:", gxf_mat.shape)
print(gxf_mat)
print()

Vector shape: (4, 1, 2, 1)
[[[[111]
   [211]]]


 [[[112]
   [212]]]


 [[[113]
   [213]]]


 [[[114]
   [214]]]]

Matrix shape: (4, 3, 2, 1)
[[[[111]
   [211]]

  [[121]
   [221]]

  [[131]
   [231]]]


 [[[112]
   [212]]

  [[122]
   [222]]

  [[132]
   [232]]]


 [[[113]
   [213]]

  [[123]
   [223]]

  [[133]
   [233]]]


 [[[114]
   [214]]

  [[124]
   [224]]

  [[134]
   [234]]]]



## `concatenate`

### For a simple test matrix

In [7]:
a = jnp.arange(5*3).reshape(5,3,1)
a

Array([[[ 0],
        [ 1],
        [ 2]],

       [[ 3],
        [ 4],
        [ 5]],

       [[ 6],
        [ 7],
        [ 8]],

       [[ 9],
        [10],
        [11]],

       [[12],
        [13],
        [14]]], dtype=int32)

In [8]:
for axis in range(2):
    ac = jnp.concatenate(a, axis=axis)
    print(f"Test matrix `a` concatenated along axis={axis}: {ac.shape}\n", ac)

Test matrix `a` concatenated along axis=0: (15, 1)
 [[ 0]
 [ 1]
 [ 2]
 [ 3]
 [ 4]
 [ 5]
 [ 6]
 [ 7]
 [ 8]
 [ 9]
 [10]
 [11]
 [12]
 [13]
 [14]]
Test matrix `a` concatenated along axis=1: (3, 5)
 [[ 0  3  6  9 12]
 [ 1  4  7 10 13]
 [ 2  5  8 11 14]]


For a more complex, though still simple matrix:

In [9]:
shape = (5,3,2,2)
a = jnp.arange(jnp.prod(jnp.asarray(shape))).reshape(shape)
a

Array([[[[ 0,  1],
         [ 2,  3]],

        [[ 4,  5],
         [ 6,  7]],

        [[ 8,  9],
         [10, 11]]],


       [[[12, 13],
         [14, 15]],

        [[16, 17],
         [18, 19]],

        [[20, 21],
         [22, 23]]],


       [[[24, 25],
         [26, 27]],

        [[28, 29],
         [30, 31]],

        [[32, 33],
         [34, 35]]],


       [[[36, 37],
         [38, 39]],

        [[40, 41],
         [42, 43]],

        [[44, 45],
         [46, 47]]],


       [[[48, 49],
         [50, 51]],

        [[52, 53],
         [54, 55]],

        [[56, 57],
         [58, 59]]]], dtype=int32)

In [10]:
for axis in range(3):
    ac = jnp.concatenate(a, axis=axis)
    print(f"Test matrix `a` concatenated along axis={axis}: {ac.shape}\n", ac)

Test matrix `a` concatenated along axis=0: (15, 2, 2)
 [[[ 0  1]
  [ 2  3]]

 [[ 4  5]
  [ 6  7]]

 [[ 8  9]
  [10 11]]

 [[12 13]
  [14 15]]

 [[16 17]
  [18 19]]

 [[20 21]
  [22 23]]

 [[24 25]
  [26 27]]

 [[28 29]
  [30 31]]

 [[32 33]
  [34 35]]

 [[36 37]
  [38 39]]

 [[40 41]
  [42 43]]

 [[44 45]
  [46 47]]

 [[48 49]
  [50 51]]

 [[52 53]
  [54 55]]

 [[56 57]
  [58 59]]]
Test matrix `a` concatenated along axis=1: (3, 10, 2)
 [[[ 0  1]
  [ 2  3]
  [12 13]
  [14 15]
  [24 25]
  [26 27]
  [36 37]
  [38 39]
  [48 49]
  [50 51]]

 [[ 4  5]
  [ 6  7]
  [16 17]
  [18 19]
  [28 29]
  [30 31]
  [40 41]
  [42 43]
  [52 53]
  [54 55]]

 [[ 8  9]
  [10 11]
  [20 21]
  [22 23]
  [32 33]
  [34 35]
  [44 45]
  [46 47]
  [56 57]
  [58 59]]]
Test matrix `a` concatenated along axis=2: (3, 2, 10)
 [[[ 0  1 12 13 24 25 36 37 48 49]
  [ 2  3 14 15 26 27 38 39 50 51]]

 [[ 4  5 16 17 28 29 40 41 52 53]
  [ 6  7 18 19 30 31 42 43 54 55]]

 [[ 8  9 20 21 32 33 44 45 56 57]
  [10 11 22 23 34 35 46 4

In [18]:
for i in range(3):
    for j in range(2):
        #ac = jnp.concatenate(jnp.moveaxis(a, 1, 2), axis=i)
        ac = jnp.concatenate(a, axis=i)
        acc = jnp.concatenate(ac, axis=j)
        #print(f"Test matrix `a` concatenated along axis={i} (shape {ac.shape}), then axis={j} (shape {acc.shape}): \n", acc)
        
        print()
        print(f"{a.shape} -- {i} --> {ac.shape} -- {j} --> {acc.shape}")
        print(acc)


(5, 3, 2, 2) -- 0 --> (15, 2, 2) -- 0 --> (30, 2)
[[ 0  1]
 [ 2  3]
 [ 4  5]
 [ 6  7]
 [ 8  9]
 [10 11]
 [12 13]
 [14 15]
 [16 17]
 [18 19]
 [20 21]
 [22 23]
 [24 25]
 [26 27]
 [28 29]
 [30 31]
 [32 33]
 [34 35]
 [36 37]
 [38 39]
 [40 41]
 [42 43]
 [44 45]
 [46 47]
 [48 49]
 [50 51]
 [52 53]
 [54 55]
 [56 57]
 [58 59]]

(5, 3, 2, 2) -- 0 --> (15, 2, 2) -- 1 --> (2, 30)
[[ 0  1  4  5  8  9 12 13 16 17 20 21 24 25 28 29 32 33 36 37 40 41 44 45
  48 49 52 53 56 57]
 [ 2  3  6  7 10 11 14 15 18 19 22 23 26 27 30 31 34 35 38 39 42 43 46 47
  50 51 54 55 58 59]]

(5, 3, 2, 2) -- 1 --> (3, 10, 2) -- 0 --> (30, 2)
[[ 0  1]
 [ 2  3]
 [12 13]
 [14 15]
 [24 25]
 [26 27]
 [36 37]
 [38 39]
 [48 49]
 [50 51]
 [ 4  5]
 [ 6  7]
 [16 17]
 [18 19]
 [28 29]
 [30 31]
 [40 41]
 [42 43]
 [52 53]
 [54 55]
 [ 8  9]
 [10 11]
 [20 21]
 [22 23]
 [32 33]
 [34 35]
 [44 45]
 [46 47]
 [56 57]
 [58 59]]

(5, 3, 2, 2) -- 1 --> (3, 10, 2) -- 1 --> (10, 6)
[[ 0  1  4  5  8  9]
 [ 2  3  6  7 10 11]
 [12 13 16 17 20 21]


**Take-away:** Our original matrix was of shape `(5, 3, 2, 2)`, which means we would like the resulting matrix to be `(5*2, 3*2) = (10, 6)`. Based on this, we should concatenate first along 1, then along 1 again.

But why? It seems like concatenating along axis $i$ somehow merges axis 0 with $i+1$. I am probably not understanding the layout of arrays.

### For tensors of the `fvv_mat` form

In [19]:
for axis in range(3):
    print(f"fvv matrix {fvv_mat.shape} concatenated along axis={axis}:", jnp.concatenate(fvv_mat, axis=axis).shape)

fvv matrix (3, 4, 2, 2) concatenated along axis=0: (12, 2, 2)
fvv matrix (3, 4, 2, 2) concatenated along axis=1: (4, 6, 2)
fvv matrix (3, 4, 2, 2) concatenated along axis=2: (4, 2, 6)


In [13]:
print("fvv concatenated once:\n", jnp.concatenate(fvv_mat, axis=1))
print("fvv concatenated twice:\n", jnp.concatenate(jnp.concatenate(fvv_mat, axis=1), axis=1))

fvv concatenated once:
 [[[1111 1121]
  [2111 2121]
  [1211 1221]
  [2211 2221]
  [1311 1321]
  [2311 2321]]

 [[1112 1122]
  [2112 2122]
  [1212 1222]
  [2212 2222]
  [1312 1322]
  [2312 2322]]

 [[1113 1123]
  [2113 2123]
  [1213 1223]
  [2213 2223]
  [1313 1323]
  [2313 2323]]

 [[1114 1124]
  [2114 2124]
  [1214 1224]
  [2214 2224]
  [1314 1324]
  [2314 2324]]]
fvv concatenated twice:
 [[1111 1121 1112 1122 1113 1123 1114 1124]
 [2111 2121 2112 2122 2113 2123 2114 2124]
 [1211 1221 1212 1222 1213 1223 1214 1224]
 [2211 2221 2212 2222 2213 2223 2214 2224]
 [1311 1321 1312 1322 1313 1323 1314 1324]
 [2311 2321 2312 2322 2313 2323 2314 2324]]


In [14]:
fvv_mat

Array([[[[1111, 1121],
         [2111, 2121]],

        [[1112, 1122],
         [2112, 2122]],

        [[1113, 1123],
         [2113, 2123]],

        [[1114, 1124],
         [2114, 2124]]],


       [[[1211, 1221],
         [2211, 2221]],

        [[1212, 1222],
         [2212, 2222]],

        [[1213, 1223],
         [2213, 2223]],

        [[1214, 1224],
         [2214, 2224]]],


       [[[1311, 1321],
         [2311, 2321]],

        [[1312, 1322],
         [2312, 2322]],

        [[1313, 1323],
         [2313, 2323]],

        [[1314, 1324],
         [2314, 2324]]]], dtype=int32)

### For the cross-covariances between the derivative observations and function values

#### For the block below the Gram matrix

In [85]:
for i in range(2):
    gfx_mat_c = jnp.concatenate(gfx_mat, axis=i)
    
    print()
    print(f"{gfx_mat.shape} -- {i} --> {gfx_mat_c.shape}")
    print(gfx_mat_c)


(4, 3, 2) -- 0 --> (12, 2)
[[111 121]
 [211 221]
 [311 321]
 [112 122]
 [212 222]
 [312 322]
 [113 123]
 [213 223]
 [313 323]
 [114 124]
 [214 224]
 [314 324]]

(4, 3, 2) -- 1 --> (3, 8)
[[111 121 112 122 113 123 114 124]
 [211 221 212 222 213 223 214 224]
 [311 321 312 322 313 323 314 324]]


In [86]:
for i in range(2):
    gfx_vec_c = jnp.concatenate(gfx_vec, axis=i)
    
    print()
    print(f"{gfx_vec.shape} -- {i} --> {gfx_vec_c.shape}")
    print(gfx_vec_c)


(4, 1, 2) -- 0 --> (4, 2)
[[111 121]
 [112 122]
 [113 123]
 [114 124]]

(4, 1, 2) -- 1 --> (1, 8)
[[111 121 112 122 113 123 114 124]]


#### For the block to the right of the Gram matrix

In [94]:
for i in range(2):
    gxf_mat_c = jnp.concatenate(gxf_mat, axis=i)
    
    print()
    print(f"{gxf_mat.shape} -- {i} --> {gxf_mat_c.shape}")
    print(gxf_mat_c)


(4, 3, 2, 1) -- 0 --> (12, 2, 1)
[[[111]
  [211]]

 [[121]
  [221]]

 [[131]
  [231]]

 [[112]
  [212]]

 [[122]
  [222]]

 [[132]
  [232]]

 [[113]
  [213]]

 [[123]
  [223]]

 [[133]
  [233]]

 [[114]
  [214]]

 [[124]
  [224]]

 [[134]
  [234]]]

(4, 3, 2, 1) -- 1 --> (3, 8, 1)
[[[111]
  [211]
  [112]
  [212]
  [113]
  [213]
  [114]
  [214]]

 [[121]
  [221]
  [122]
  [222]
  [123]
  [223]
  [124]
  [224]]

 [[131]
  [231]
  [132]
  [232]
  [133]
  [233]
  [134]
  [234]]]


In [95]:
for i in range(2):
    gxf_vec_c = jnp.concatenate(gxf_vec, axis=i)
    
    print()
    print(f"{gxf_vec.shape} -- {i} --> {gxf_vec_c.shape}")
    print(gxf_vec_c)


(4, 1, 2, 1) -- 0 --> (4, 2, 1)
[[[111]
  [211]]

 [[112]
  [212]]

 [[113]
  [213]]

 [[114]
  [214]]]

(4, 1, 2, 1) -- 1 --> (1, 8, 1)
[[[111]
  [211]
  [112]
  [212]
  [113]
  [213]
  [114]
  [214]]]


## `reshape`

**Take-away:** I don't think this can be done.

Back to the simpler 4D array:

In [20]:
shape = (5,3,2,2)
a = jnp.arange(jnp.prod(jnp.asarray(shape))).reshape(shape)
a

Array([[[[ 0,  1],
         [ 2,  3]],

        [[ 4,  5],
         [ 6,  7]],

        [[ 8,  9],
         [10, 11]]],


       [[[12, 13],
         [14, 15]],

        [[16, 17],
         [18, 19]],

        [[20, 21],
         [22, 23]]],


       [[[24, 25],
         [26, 27]],

        [[28, 29],
         [30, 31]],

        [[32, 33],
         [34, 35]]],


       [[[36, 37],
         [38, 39]],

        [[40, 41],
         [42, 43]],

        [[44, 45],
         [46, 47]]],


       [[[48, 49],
         [50, 51]],

        [[52, 53],
         [54, 55]],

        [[56, 57],
         [58, 59]]]], dtype=int32)

In [30]:
#print(a.reshape(15, 2, 2)) # Corresponds to concatenation along axis 0
print(a.reshape(3, 10, 2)) # Does NOT orrespond to concatenation along axis 1
#print(a.reshape(3, 2, 10)) # Does NOT correspond to concatenation along axis 2

[[[ 0  1]
  [ 2  3]
  [ 4  5]
  [ 6  7]
  [ 8  9]
  [10 11]
  [12 13]
  [14 15]
  [16 17]
  [18 19]]

 [[20 21]
  [22 23]
  [24 25]
  [26 27]
  [28 29]
  [30 31]
  [32 33]
  [34 35]
  [36 37]
  [38 39]]

 [[40 41]
  [42 43]
  [44 45]
  [46 47]
  [48 49]
  [50 51]
  [52 53]
  [54 55]
  [56 57]
  [58 59]]]


In [33]:
print(a.reshape(3, 10, 2).reshape(10,6))

[[ 0  1  2  3  4  5]
 [ 6  7  8  9 10 11]
 [12 13 14 15 16 17]
 [18 19 20 21 22 23]
 [24 25 26 27 28 29]
 [30 31 32 33 34 35]
 [36 37 38 39 40 41]
 [42 43 44 45 46 47]
 [48 49 50 51 52 53]
 [54 55 56 57 58 59]]


In [36]:
print(a.reshape(3,10,2).)

[[[ 0  1]
  [ 2  3]
  [ 4  5]
  [ 6  7]
  [ 8  9]
  [10 11]
  [12 13]
  [14 15]
  [16 17]
  [18 19]]

 [[20 21]
  [22 23]
  [24 25]
  [26 27]
  [28 29]
  [30 31]
  [32 33]
  [34 35]
  [36 37]
  [38 39]]

 [[40 41]
  [42 43]
  [44 45]
  [46 47]
  [48 49]
  [50 51]
  [52 53]
  [54 55]
  [56 57]
  [58 59]]]


# Testing an actual covariance function for derivative information

In [94]:
def eq(lengthscale, variance):
    "The exponentiated quadratic covariance function."

    def k(x, y):
        return variance * jnp.exp(-0.5 * jnp.sum(((x - y) / lengthscale) ** 2))

    return k

# The lazy way by GPJax:
def cov_matrix(kernel, x, y):
    K = jax.vmap(lambda x: jax.vmap(lambda y: kernel(x, y))(y))(x)
    return K


In [96]:
# Test data

xx = jnp.arange(1., 1. + 5 * 2).reshape(5,2)
yy = jnp.arange(1., 1. + 5 * 2).reshape(5,2)

## The Gram matrix

In [97]:
k = eq(1, 1)
d2dxdy = jax.jacfwd(jax.jacrev(k, argnums=0), argnums=1)

In [98]:
d2dxdy(xx[0], yy[0])

Array([[1., 0.],
       [0., 1.]], dtype=float32)

In [99]:
d2dxdy_cov = vmap(
    vmap(d2dxdy, in_axes=(0, None), out_axes=0),
    in_axes=(None, 0), out_axes=1
)

d2dxdy_mat_a = d2dxdy_cov(xx, yy)
d2dxdy_mat_b = cov_matrix(d2dxdy, xx, yy)

print(d2dxdy_cov(xx, yy).shape)

print(jnp.concatenate(jnp.concatenate(d2dxdy_cov(xx, yy), axis=1), axis=1).shape)

(5, 5, 2, 2)
(10, 10)


In [100]:
jnp.allclose(d2dxdy_mat_a, d2dxdy_mat_b)

Array(True, dtype=bool)

## The cross-covariances

In [107]:
ddx = jax.jacfwd(k, argnums=0)
ddy = jax.jacfwd(k, argnums=1)

# To the right of the Gram matrix:
ddx_cov = vmap(
    vmap(ddx, in_axes=(0, None), out_axes=0),
    in_axes=(None, 0), out_axes=2
)
# below the Gram matrix:
ddy_cov = vmap(
    vmap(ddy, in_axes=(0, None), out_axes=0),
    in_axes=(None, 0), out_axes=0
)

ddx_mat = ddx_cov(xx, yy[:1])
ddy_mat = ddy_cov(xx[:1], yy)

ddx_mat_b = cov_matrix(ddx, xx, yy[:1])
ddy_mat_b = cov_matrix(ddy, xx[:1], yy)

print(ddx_mat.shape)
print(ddy_mat.shape)

ddx_c = jnp.concatenate(ddx_mat, axis=0)
ddy_c = jnp.concatenate(ddy_mat, axis=1)

print(jnp.concatenate(ddx_mat, axis=0).shape)
print(jnp.concatenate(ddx_mat, axis=1).shape)
jnp.concatenate(ddy_mat, axis=1).shape

(5, 2, 1)
(5, 1, 2)
(10, 1)
(2, 5)


(1, 10)

In [108]:
print(jnp.allclose(ddx_mat, ddx_mat_b))
print(jnp.allclose(ddy_mat, ddy_mat_b))

True
False


In [115]:
print(jnp.concatenate(ddy_mat, axis=1))

[[-0.00000000e+00 -0.00000000e+00 -3.66312787e-02 -3.66312787e-02
  -4.50140703e-07 -4.50140703e-07 -1.39171367e-15 -1.39171367e-15
  -1.28304875e-27 -1.28304875e-27]]


In [121]:
print(jnp.concatenate(ddy_mat_b, axis=1))

[[-0.00000000e+00 -0.00000000e+00]
 [-3.66312787e-02 -3.66312787e-02]
 [-4.50140703e-07 -4.50140703e-07]
 [-1.39171367e-15 -1.39171367e-15]
 [-1.28304875e-27 -1.28304875e-27]]


In [118]:
ddx_mat
jnp.concatenate(ddx_mat, axis=0)

Array([[-0.00000000e+00],
       [-0.00000000e+00],
       [-3.66312787e-02],
       [-3.66312787e-02],
       [-4.50140703e-07],
       [-4.50140703e-07],
       [-1.39171367e-15],
       [-1.39171367e-15],
       [-1.28304875e-27],
       [-1.28304875e-27]], dtype=float32)

In [47]:
ddy_mat
jnp.concatenate(ddy_mat, axis=1)

Array([[-0.00000000e+00, -0.00000000e+00, -3.66312787e-02,
        -3.66312787e-02, -4.50140703e-07, -4.50140703e-07,
        -1.39171367e-15, -1.39171367e-15, -1.28304875e-27,
        -1.28304875e-27]], dtype=float32)

In [43]:
ddx_c.T == ddy_c

Array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True]], dtype=bool)

In [61]:
xx[:, :1].ravel()[:,None]

Array([[1.],
       [3.],
       [5.],
       [7.],
       [9.]], dtype=float32)

In [63]:
jnp.atleast_2d(xx[:, :1].ravel()).T

Array([[1.],
       [3.],
       [5.],
       [7.],
       [9.]], dtype=float32)

In [68]:
k = eq(1, 1)
vmap(lambda x: vmap(lambda y: k(x, y))(yy[:, :1]))(xx[:, :1])

Array([[1.0000000e+00, 1.3533528e-01, 3.3546262e-04, 1.5229979e-08,
        1.2664166e-14],
       [1.3533528e-01, 1.0000000e+00, 1.3533528e-01, 3.3546262e-04,
        1.5229979e-08],
       [3.3546262e-04, 1.3533528e-01, 1.0000000e+00, 1.3533528e-01,
        3.3546262e-04],
       [1.5229979e-08, 3.3546262e-04, 1.3533528e-01, 1.0000000e+00,
        1.3533528e-01],
       [1.2664166e-14, 1.5229979e-08, 3.3546262e-04, 1.3533528e-01,
        1.0000000e+00]], dtype=float32)

In [ ]:
vmap(vmap(k))